# Python으로 메일 보내기

SMTP: 메일 약속 프로토콜(간단하게 메일을 보내기 위한 약속)

파이썬 프로그램에서 이메일을 만들고 SMTP로 이메일로 전송
-> a 이메일 서버를 b라는 이메일 서버에 보낼 때 SMTP 사용
  (반대도 가능)

IMAP으로 다른 이메일이 우리 프로그램으로 가져올 수 있다.

(b-> smtp를 이용해 b@gmail.com에 전송 -> smtp로 a@gamil.com에게 전송 -> imap을 이용해 a email client에게 전송)

원래는 email server끼리 smtp로 주고받음

(email client가 python)

smtp서버를 이용해서 우리가 원하는 곳으로 메일을 보낼 수 있다.

**text 메일 보내기**

In [ ]:
# 서버에서 어떤 문을 열고 갈지 결정이 port
# import smtplib -> smtp 사용
# from email.message import EmailMessage

# SMTP_SERVER="smtp.gmail.com"
# SMTP_PORT=465

# smtp=smtplib.SMTP(SMTP_SERVER,SMTP_PORT) -> 서버 연결해주는 명령 -> 서버 주소와 포트번호를 알아야한다.
# print(smtp) -> SSL(암호화방식)이 없어서 연결을 하지 못한다.(이렇게 하면) -> 암호화 방식이 있는 함수로 연결해야한다. -> 연결

# smtp=smtplib.SMTP_SSL(SMTP_SERVER,SMTP_PORT) -> SSL을 요구하는 곳에 접근하기 위해서는 이렇게 수정

# smtp.login("j961224@likelion.org","88697170") -> 아이디와 비밀번호가 parameter이다.

#----------------------------------------------------------------#

#그냥 말을 넣어서 보내면 smtp는 못 알아들음. 그래서 text파일을 만들어서 mime형태로 바꿔서 보내줘야한다.

# message = EmailMessage()
# message.set_content("코드라이언 수업중입니다.") ->mime형태로 만들어줌

# message["Subject"] = "제목"
# message["From"]="발신인"
# message["To"]="수신인"       ----> subject from to 같은 경우 mime 형태로 만들 경우 header로 들어가므로

import smtplib
from email.message import EmailMessage

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

message = EmailMessage()
message.set_content("코드라이언 수업중입니다.")

message["Subject"] = "이것은 제목입니다."
message["From"] = "###@gmail.com"
message["To"] = "###@gmail.com"

smtp = smtplib.SMTP_SSL(SMTP_SERVER,SMTP_PORT)
smtp.login("###@gmail.com","######")
smtp.send_message(message) #-> 메세지 보내는 코드
smtp.quit()
#-> 메일 보내기


**사진 첨부된 메일 보내기**

* 메일에 사진 첨부하기

rb -> read binary

wb -> write binary

ab -> append binary

In [ ]:
# image = open("codelion.png","rb")
# print(image.read()) -> 이미지가 무슨 이상한 문자로 읽힘

# with open("codelion.png","rb") as image: -> close문 없이 자동으로 끝나면 닫힘 -> readbinary모드로 읽어서 이미지로 읽음 
#     image_file=image.read() -> 이미지 파일 변수 안에 이미지를 읽은 내용을 담아준다.

# -> 이것을 프린트하면 binary들이 출력된다. -> 앞에꺼보다 조금 더 안전하다(자동으로 닫혀서?)

# add_attachment(..) => text가 아닌 다른 파일을 첨부하기 위해서 사용한다.
# mixed type -> text 뿐만 아니라 다른 파일도 섞인 것을 말한다.

# message.add_attachment(image_file,maintype='image',subtype='png') -> image,maintype,subtype

# 1. image
# 2. maintype 메인 이미지같은 것
# 3.subtype은 확장자

In [1]:
# 자동으로 파일 확장자 파악하여 메일 보내는 코드!

import smtplib
from email.message import EmailMessage
import imghdr

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

message = EmailMessage()
message.set_content("코드라이언 수업중입니다.")

message["Subject"] = "이것은 제목입니다."
message["From"] = "###@gmail.com"
message["To"] = "###@gmail.com"

with open("codelion.png","rb") as image:
    image_file = image.read()

image_type = imghdr.what('codelion',image_file) #-> 파일명,실제 파일 데이터 넣기
print(image_type) # -> png가 나옴(동적으로 이미지 타입을 가져온다)
message.add_attachment(image_file,maintype='image',subtype=image_type)

smtp = smtplib.SMTP_SSL(SMTP_SERVER,SMTP_PORT)
smtp.login("###@gmail.com","######")
smtp.send_message(message)
smtp.quit()

In [ ]:
#이메일 보내기 (python으로) -> 메일 유효성 체크

import smtplib
from email.message import EmailMessage
import imghdr
import re #-> 이것으로 정규표현식을 쓰도록 가능하게 함

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

def sendEmail(addr):
    #이메일 주소 유효성 체크하기(보내려고 하는 이메일 주소가 있는건지 확인)
    #--> 정규표현식으로(문자열에서 나타나는 특정 패턴을 알아내서 대응시키기위해 사용) -> 이메일의 특정 패턴을 조건으로 주고 이 조건의 문자열이 적합한지 판단
    #첫번째 [a-zA-Z0-9.+_-]+는 a부터 z까지, A부터 Z까지, 0부터 9까지, .,+,_,-가 1회 이상 반복(마지막 + 때문에 반복)된다.
    #다음으로 @는 그 뒤에 @가 붙어야한다는 의미
    #다음은 [a-zA-Z0-9]+ a부터 z까지, A부터 Z까지, 0부터 9까지 1회이상 반복된다.
    #다음으로 \.는 실제 그 뒤에 .이 붙도록한다. -> 일반 문자로써의 .을 의미해서 \를 붙임
    #다음으로 [a-zA-Z]{2,3} a부터 z까지, A부터 Z까지가 최소 3회, 최대 3번 반복된다.
    #처음과 끝에는 ^와$를 붙인다.
    reg = "^[a-zA-Z0-9.+_-]+@[a-zA-Z0-9]+\.[a-zA-Z]{2,3}$"
    
    if bool(re.match(reg,addr)):
        smtp.send_message(message)
        print("정상적으로 메일이 발송되었습니다.")
    else:
        print("유효한 이메일 주소가 아닙니다.")

message = EmailMessage()
message.set_content("코드라이언 수업중입니다.")

message["Subject"] = "이것은 제목입니다."
message["From"] = "###@gmail.com"
message["To"] = "###@gmail.com"

with open("codelion.png","rb") as image:
    image_file = image.read()

image_type = imghdr.what('codelion',image_file)
message.add_attachment(image_file,maintype='image',subtype=image_type)

smtp = smtplib.SMTP_SSL(SMTP_SERVER,SMTP_PORT)
smtp.login("###@gmail.com","######")
# 메일을 보내는 sendEmail 함수를 호출해서 실행해보기
sendEmail("###gmailcom")
smtp.quit()